In [1]:
library(dplyr)
library(dummies)
library(ROCR)
library(Metrics)
library(caTools)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

dummies-1.5.6 provided by Decision Patterns

Warning message:
"package 'ROCR' was built under R version 3.4.2"Loading required package: gplots
Warning message:
"package 'gplots' was built under R version 3.4.2"
Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess

Warning message:
"package 'caTools' was built under R version 3.4.2"

In [2]:
df<-read.csv("Car accidents.csv")
df<-na.omit(df)
df$X <- NULL #inutil
df$caseid<-NULL  #inutil
df$dvcat<- as.character(df$dvcat)
df$dvcat[df$dvcat == "1-9km/h"] <- "1-9"
df$dvcat <- as.factor(df$dvcat)
df$dead<- ifelse(df$dead == "alive",0,1)
df$seatbelt<- ifelse(df$seatbelt == "belted",1,0)
df$occRole<- ifelse(df$occRole == "driver",1,0)
df$sex<-ifelse(df$sex == "m",1,0)

In [3]:
wghts<-df$weight
df<-df%>%
  select(-c(weight,airbag))

**_dead_  - se a vítima morreu ou não.**

_dvcat_ - velocidade do impacto estimado em km/h. **Dummy Variable** 
 - 1-9 
 - 10-24
 - 25-39
 - 40-54
 - 55+

_seatbelt_ - se a vítima observada estava com cinto ou não. 

_frontal_ - se o impacto da observação foi frontal ou não.

_ageOFocc_ - a idade da pessoa envolvida no acidente em anos.

_abcat_ - um ou mais airbags foram ativados(seja motorista ou passageiro)?.
 - deploy
 - no deploy 
 - unavail

_deploy_ - mesmo que abcat_deploy. **retiramos** 

_airbag_ - inverso de abcat_unavail. **retiramos** 

_injSeverity_ - severidade do ferimento da vítima observada, numérico. ** Dummy Variable**
 - 0: nenhum 
 - 1: possivelmente ferido 
 - 2: sem incapacitação no momento após o acidente
 - 3: incapacitado 
 - 4: morto **mesmo que _dead_**
 - 5: desconhecido
 - 6: morte anterior (ao acidente). 
 
_weight_ - uma estimativa do inverso da probabilidade de seleção (Sample weights). **separamos a um vetor**

_sex_ - sexo da vítima observada (f,m).

_yearacc_ - ano em que o acidente ocorreu.

_yearVeh_ - ano do modelo do carro. 

_occRole_ - papel do ocupante da observação (motorista ou passageiro).<br>

In [4]:
dumf<-dummy.data.frame(df,names=c('dvcat'),sep ='_')
dumf<-dummy.data.frame(dumf,names=c('abcat'),sep='_')

In [5]:
`2x2chiSq`<-table(dumf$dead,dumf$seatbelt) 
chisq.test(`2x2chiSq`)


	Pearson's Chi-squared test with Yates' continuity correction

data:  2x2chiSq
X-squared = 483.76, df = 1, p-value < 2.2e-16


$$log(\frac{P_e}{1-P_e}) = \beta_0 + \beta_1X_1 + \beta_2X_2 ...$$

```R
glm(y~intercept, family="binomial") vs. glm(y~variable, family="binomial")
```

<img src="LRT arredondado.png">

$$ D=2\cdot(LL_{alternativo} - LL_{intercepto (nulo)})\sim \chi_{df1-df2}^{2}$$